# Manoeuver Generation Workflow

In [ ]:
from fds.config import set_api_key, set_url
set_url('https://api.spacetower.exotrail.space/fds/v1')
set_api_key('MY_API_KEY')

In [ ]:
import pandas as pd
import plotly.express as px
from fds.models.maneuvers.strategy import ManeuverStrategy
from fds.models.maneuvers.use_case import ManeuverGeneration
from fds.models.orbital_state import PropagationContext, CovarianceMatrix, OrbitalState
from fds.models.spacecraft import Battery, SolarArray, ThrusterElectrical, SpacecraftBox
from fds.models.two_line_element import TwoLineElement
from fds.utils.frames import Frame


In [ ]:
latest_tle = TwoLineElement(
    "1 58295U 23174AR  24137.17541737  .00005775  00000-0  29855-3 0  9994",
    "2 58295  97.4580 211.9750 0010360 352.3995   7.7079 15.16607347 28836"
)

propagation_context = PropagationContext(
    model_perturbations=[
        PropagationContext.Perturbation.DRAG,
        PropagationContext.Perturbation.EARTH_POTENTIAL,
        PropagationContext.Perturbation.SRP,
        PropagationContext.Perturbation.THIRD_BODY,
    ],
    model_solar_flux=150,  # SFU
    model_earth_potential_deg=10,
    model_earth_potential_ord=10,
    model_atmosphere_kind=PropagationContext.AtmosphereModel.HARRIS_PRIESTER,
    integrator_kind=PropagationContext.IntegratorKind.DORMAND_PRINCE_853,
    integrator_min_step=0.01,  # s
    integrator_max_step=100,  # s
)

battery = Battery(
    depth_of_discharge=0.3,  # 0<x<1
    nominal_capacity=560,  # W
    minimum_charge_for_firing=0.9,  # 0<x<1
    initial_charge=1,  # 0<x<1
)

solar_array = SolarArray(
    kind="DEPLOYABLE_FIXED",
    initialisation_kind=SolarArray.InitialisationKind.MAXIMUM_POWER,
    efficiency=.293,  # 0<x<1
    normal_in_satellite_frame=(0, 0, -1),  # Unit vector
    maximum_power=300,  # W
    surface=0.7490056949399673,  # m^2
)

electrical_thruster = ThrusterElectrical(
    isp=950,  # s
    thrust=0.005,  # N
    axis_in_satellite_frame=(-1, 0, 0),  # Unit vector
    propellant_mass=4,  # kg
    wet_mass=11,  # kg
    warm_up_duration=240,  # s
    maximum_thrust_duration=1200,  # s
    impulse=37265.27,  # Ns
    power=300,  # W
    stand_by_power=1.1,  # W
    warm_up_power=50,  # W
)

spacecraft = SpacecraftBox(
    battery=battery,
    thruster=electrical_thruster,
    solar_array=solar_array,
    platform_mass=112,  # kg
    drag_coefficient=2.2,
    length_x=.5,  # m
    length_y=.5,  # m
    length_z=.5,  # m
    max_angular_velocity=2,  # deg/s
    max_angular_acceleration=.5,  # deg/s^2
)

covariance = CovarianceMatrix.from_diagonal(
    diagonal=(100, 100, 100, 0.1, 0.1, 0.1),
    frame=Frame.TNW
)

orbital_state = OrbitalState.from_tle(
    tle=latest_tle,
    covariance_matrix=covariance,
    propagation_context=propagation_context,
    spacecraft=spacecraft,
)

## Executing the use case

In [ ]:
maneuver_strategy = ManeuverStrategy(
    thrust_arcs_position="ASCENDING_AND_DESCENDING_NODES",
    thrust_arcs_number="ONE",
    thrust_arc_initialisation_kind="DUTY_CYCLE",
    number_of_thrust_orbits=1,
    number_of_rest_orbits=2,
    number_of_shift_orbits=1,
    orbital_duty_cycle=0.2,
    thrust_arc_duration=1200,
    stop_thrust_at_eclipse=False
)

mg = ManeuverGeneration(
    initial_orbital_state=orbital_state,
    strategy=maneuver_strategy,
    delta_semi_major_axis=.2,  # km
    delta_eccentricity=0,
    delta_inclination=0.0,
    quaternion_step=60,
    maximum_duration=10 * 24 * 60 * 60  # seconds
)
res_mg = mg.run().result

## Plotting the results

In [ ]:
c_df, d_df = res_mg.export_roadmap_data_for_dataframe()
c_g_th, d_g_th = res_mg.export_thruster_data_for_gantt()
c_g_at, d_g_at = res_mg.export_attitude_data_for_gantt()

df_all = pd.DataFrame(d_df, columns=c_df)
df_all

In [ ]:
df_thruster = pd.DataFrame(d_g_th, columns=c_g_th)
df_attitude = pd.DataFrame(d_g_at, columns=c_g_at)

fig = px.timeline(df_thruster, x_start="Start", x_end="End", color="Mode", y="Mode")
fig.show()

fig = px.timeline(df_attitude, x_start="Start", x_end="End", color="Mode", y="Mode")
fig.show()